In [1]:
import cv2
import numpy as np
import random
import h5py as h5
from pathlib import Path
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Input
from keras.models import Model
from keras.optimizers import Adam
from keras.utils import plot_model
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
from tensorflow.keras.utils import plot_model
from tqdm import tqdm

# Training the F-RCNN

## Importing auxiliary functions

In [2]:
import sys
sys.path.append("..") # TODO: fix

In [3]:
from training.read_dataset import read_dataset
from training.create_anchors import create_anchors
from training.calculate_bbox_intesect_over_union import calculate_bbox_intesect_over_union
from training.evaluate_ious import evaluate_ious
from training.create_samples_for_training import create_samples_for_training
from training.parametrize_anchor_box_properties import parametrize_anchor_box_properties
from training.losses import loss_cls, loss_reg
from training.run import generate_validation_data
from training.run import input_generator

## Defining the path of the dataset

In [4]:
dataset_folder = Path("dataset_test")

## Reading the dataset

In [5]:
IMG_SIZE = (500, 500)
imgs, bbox_datasets = read_dataset(IMG_SIZE, dataset_folder)

## Shuffling the data

In [6]:
RANDOM_SEED = 13

In [7]:
# Shuffling dataset
N_dataset = imgs.shape[0]
array_for_shuffling = np.arange(N_dataset, dtype=int)
random.Random(RANDOM_SEED).shuffle(array_for_shuffling)

# Shuffling images
imgs = imgs[array_for_shuffling]

# Shuffling bboxes
bbox_datasets_new = []
for new_index in array_for_shuffling:
    bbox_datasets_new.append(bbox_datasets[new_index])
bbox_datasets = bbox_datasets_new

In [8]:
# Defining which image is used during training
MODE = "mask"
if MODE == "mask":
    imgs = imgs[:,:,:,0]
elif MODE == "raw":
    imgs = imgs[:,:,:,1]

## Defining some parameters

In [9]:
N_SUB = 8
ANCHOR_REAL_SIZE = [16, 24, 32, 48, 64]
POS_IOU_THRESHOLD = 0.50
NEG_IOU_THRESHOLD = 0.1
N_FILTERS = 16
KERNEL_SIZE = 5

ANCHOR_RATIOS = [0.5, 0.8, 1.0, 1.1]

SHOW_N_POS = False
POS_RATIO = 0.5
N_SAMPLES = 30

ADAM_LEARNING_RATE = 3.0e-4

# Subscaling/Anchor values (2^n), ex: 1, 2, 4, 8, 16, 32
N_SUB = 2

# Defining anchor sizes
ANCHOR_SIZES = np.array(ANCHOR_REAL_SIZE) // N_SUB

# Defining number of anchors sized and rations
N_ANCHORS =  len(ANCHOR_SIZES)
N_RATIOS  =  len(ANCHOR_RATIOS)

## Defining the F-RCNN architecture

In [10]:
input_image = Input(shape=(IMG_SIZE[0],IMG_SIZE[1],1))

conv_3_3_1 = Conv2D(
    filters=N_FILTERS,
    kernel_size=KERNEL_SIZE,
    padding='same',
    name="3x3-1"
    )(input_image)
max_pool_1 = MaxPooling2D((2,2),
                          name="max_pool_1")(conv_3_3_1)

conv_3_3_2 = Conv2D(
    filters=N_FILTERS,
    kernel_size=KERNEL_SIZE,
    padding='same',
    name="3x3-2"
)(max_pool_1)


max_pool_2 = MaxPooling2D((2,2),
                          name="max_pool_2")(conv_3_3_2)

conv_3_3_3 = Conv2D(
    filters=N_FILTERS,
    kernel_size=KERNEL_SIZE,
    padding='same',
    name="3x3-3"
)(max_pool_2)

max_pool_3 = MaxPooling2D((2,2),
                          name="max_pool_3")(conv_3_3_3)

conv_3_3_4 = Conv2D(
    filters=N_FILTERS,
    kernel_size=KERNEL_SIZE,
    padding='same',
    name="3x3-4"
)(max_pool_3)

max_pool_4 = MaxPooling2D((2,2),
                          name="max_pool_4")(conv_3_3_4)

conv_3_3_5 = Conv2D(
    filters=N_FILTERS,
    kernel_size=KERNEL_SIZE,
    padding='same',
    name="3x3-5"
)(max_pool_4)

max_pool_5 = MaxPooling2D((2,2),
                          name="max_pool_5")(conv_3_3_5)

conv_3_3_6 = Conv2D(
    filters=N_FILTERS,
    kernel_size=KERNEL_SIZE,
    padding='same',
    name="3x3-6"
)(max_pool_5)

max_pool_6 = MaxPooling2D((2,2),
                          name="max_pool_6")(conv_3_3_6)

In [11]:
N_SUB = 8

if N_SUB == 1:
    last_layer = conv_3_3_1
elif N_SUB == 2:
    last_layer = max_pool_1
elif N_SUB == 4:
    last_layer = max_pool_2
elif N_SUB == 8:
    last_layer = max_pool_3
elif N_SUB == 16:
    last_layer = max_pool_4
elif N_SUB == 32:
    last_layer = max_pool_5

In [12]:
output_scores = Conv2D(
    filters=N_ANCHORS * N_RATIOS,
    kernel_size=(1, 1),
    activation="sigmoid",
    kernel_initializer="uniform",
    name="l_reg"
)(last_layer)

output_regressor = Conv2D(
    filters=N_ANCHORS * N_RATIOS * 4,
    kernel_size=(1, 1),
    activation="linear",
    kernel_initializer="uniform",
    name="bb_reg"
)(last_layer)

opt = Adam(learning_rate=ADAM_LEARNING_RATE)
model = Model(inputs=[input_image], outputs=[output_scores, output_regressor])
model.compile(optimizer=opt, loss={'l_reg':loss_cls, 'bb_reg':loss_reg})

Below we can get a summary of the model. Alternatively (and recommended), you can check the F-RCNN architecture with Keras' `plot_model` function (see [here](https://keras.io/api/utils/model_plotting_utils/)).

In [13]:
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 500, 500, 1) 0                                            
__________________________________________________________________________________________________
3x3-1 (Conv2D)                  (None, 500, 500, 16) 416         input_1[0][0]                    
__________________________________________________________________________________________________
max_pool_1 (MaxPooling2D)       (None, 250, 250, 16) 0           3x3-1[0][0]                      
__________________________________________________________________________________________________
3x3-2 (Conv2D)                  (None, 250, 250, 16) 6416        max_pool_1[0][0]                 
_______________________________________________________________________________________

## Training the model

Now, we need to setup a few callbacks to be executed during training.

The first one is the `early_stopping` callback, which defines a `PATIENCE` value to limit the number of running epochs (`EPOCHS`).

The second one, `best_model`, is a callback to store the F-RCNN with the lowest validation loss.

In [14]:
N_EPOCHS = 100
N_PATIENCE = 200

In [15]:
N_VALIDATION_DATA = 10
N_DATA_EPOCHS = 10
DEBUG = False

In [16]:
# Defining the best fRCNN model name
best_model_name = "best_fRCNN_%s_%02d.keras" % (MODE, N_SUB)

early_stopping = EarlyStopping(monitor='val_loss',
                               mode='min',
                               verbose=1,
                               patience=N_PATIENCE)

best_model = ModelCheckpoint(best_model_name,
                              verbose=1,
                              save_best_only=True,
                              monitor='val_loss',
                              mode='auto')

On this function below, it will generate the validation data necessary to train the model.

In [17]:
validation_data = generate_validation_data(imgs[:N_VALIDATION_DATA],
                                           bbox_datasets[:N_VALIDATION_DATA],
                                           IMG_SIZE,
                                           N_SUB,
                                           N_ANCHORS,
                                           ANCHOR_SIZES,
                                           ANCHOR_RATIOS,
                                           N_RATIOS,
                                           POS_IOU_THRESHOLD,
                                           NEG_IOU_THRESHOLD,
                                           DEBUG,
                                           POS_RATIO,
                                           N_SAMPLES,
                                           SHOW_N_POS)

Generating validation data: 10it [00:01,  6.43it/s]


Now, it's time to train the model with our data.

In [18]:
model.fit(input_generator(imgs[N_VALIDATION_DATA:],
                          bbox_datasets[N_VALIDATION_DATA:],
                          IMG_SIZE,
                          N_SUB,
                          ANCHOR_RATIOS,
                          ANCHOR_SIZES,
                          N_DATA_EPOCHS,
                          N_ANCHORS,
                          N_RATIOS,
                          POS_IOU_THRESHOLD,
                          NEG_IOU_THRESHOLD,
                          DEBUG, 
                          POS_RATIO,
                          N_SAMPLES,
                          SHOW_N_POS),
          steps_per_epoch=10,
          epochs=N_EPOCHS,
          callbacks=[best_model, early_stopping],
          validation_data = validation_data)

[15  5 11  5 12  8 16  5  0 10]
Epoch 1/100
[15  5 11  5 12  8 16  5  0 10]
10/10 [==============================] - ETA: 0s - loss: 0.7137 - l_reg_loss: 0.6922 - bb_reg_loss: 0.0215    
Epoch 00001: val_loss improved from inf to 0.92446, saving model to best_fRCNN_mask_08.keras
10/10 [==============================] - 10s 1s/step - loss: 0.7137 - l_reg_loss: 0.6922 - bb_reg_loss: 0.0215 - val_loss: 0.9245 - val_l_reg_loss: 0.6918 - val_bb_reg_loss: 0.2326
Epoch 2/100
[ 2  1  8  6 12  6  1  1 14  0]
10/10 [==============================] - ETA: 0s - loss: 0.7096 - l_reg_loss: 0.6893 - bb_reg_loss: 0.0203    
Epoch 00002: val_loss improved from 0.92446 to 0.91236, saving model to best_fRCNN_mask_08.keras
10/10 [==============================] - 10s 1s/step - loss: 0.7096 - l_reg_loss: 0.6893 - bb_reg_loss: 0.0203 - val_loss: 0.9124 - val_l_reg_loss: 0.6890 - val_bb_reg_loss: 0.2234
Epoch 3/100
[ 4  7  4  1  2  0  2 12 16  3]
 2/10 [=====>........................] - ETA: 4s - loss: 0.686


KeyboardInterrupt



At this point, the model is trained. In order to use the model as a segmentation tool, please check the `predicting_example`.
